# Collect Datasets
First we look at the discipline page.

## Expulsion

In [1]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import os

In [2]:
url = "https://www.cde.ca.gov/ds/ad/filesed.asp"
html = requests.get(url).text
soup = BeautifulSoup(html,'html')

In [3]:
table = soup.find("table")
files = {}
for j,row in enumerate(table.find_all("tr")):
    if j==0: continue #Skip header of the table
    date = None
    file_path = None
    for i,cell in enumerate(row.find_all("td")):
        if i==0:
            #Get first column as the date
            date = cell.text.strip()
        elif i==1:
            file_path = cell.find('a').attrs['href']
    files[date]=file_path
    print(date,file_path)

2020–21 https://www3.cde.ca.gov/demo-downloads/discipline/exp2021.txt
2019–20 https://www3.cde.ca.gov/demo-downloads/discipline/exp1920.txt
2018–19 https://www3.cde.ca.gov/demo-downloads/discipline/exp1819.txt
2017–18 https://www3.cde.ca.gov/demo-downloads/discipline/exp1718.txt
2016–17 https://www3.cde.ca.gov/demo-downloads/discipline/exp1617.txt
2015–16 https://www3.cde.ca.gov/demo-downloads/discipline/exp1516.txt
2014–15 https://www3.cde.ca.gov/demo-downloads/discipline/exp1415.txt
2013–14 https://www3.cde.ca.gov/demo-downloads/discipline/exp1314.txt
2012–13 https://www3.cde.ca.gov/demo-downloads/discipline/exp1213.txt
2011–12 https://www3.cde.ca.gov/demo-downloads/discipline/exp1112.txt


Now download them all

use pandas to put them into dataframes

In [4]:
texts = [requests.get(file_path).text for _,file_path in files.items()]

Turn them to dataframes, I have to build the strucutre myself though... parsing.

In [10]:
#Given the text from the file
#Return an an array of arrays that pandas can read. 
def clean(string):
    lines = string.split('\n')
    clean_line = lambda line: line.replace("ï»¿","").strip()
    return [clean_line(line).split('\t') for line in lines]

In [11]:
dataframes = []
for text in texts:
    data = clean(text)
    dataframes.append(pd.DataFrame(data[1:],columns=data[0]))

In [12]:
df = pd.concat(dataframes)

In [13]:
df.to_csv("expulsion.csv")
df

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,...,Unduplicated Count of Students Expelled (Total),Unduplicated Count of Students Expelled (Defiance-Only),Expulsion Rate (Total),Expulsion Count Violent Incident (Injury),Expulsion Count Violent Incident (No Injury),Expulsion Count Weapons Possession,Expulsion Count Illicit Drug-Related,Expulsion Count Defiance-Only,Expulsion Count Other Reasons,Cumulative Enrollment
0,2020-21,C,01,,,Alameda,,,All,GF,...,1,0,0.0,0,0,0,0,0,1,NaN
1,2020-21,C,01,,,Alameda,,,All,GM,...,0,0,0.0,0,0,0,0,0,0,NaN
2,2020-21,C,01,,,Alameda,,,All,GX,...,0,0,0.0,0,0,0,0,0,0,NaN
3,2020-21,C,01,,,Alameda,,,All,RA,...,1,0,0.0,0,0,0,0,0,1,NaN
4,2020-21,C,01,,,Alameda,,,All,RB,...,0,0,0.0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345156,2011-12,T,00,,,State,,,No,SS,...,7127,395,0.2,1235,1832,1160,2308,400,345,3611833
345157,2011-12,T,00,,,State,,,No,TA,...,9335,483,0.2,1542,2255,1543,3265,488,446,5890875
345158,2011-12,T,00,,,State,,,Yes,TA,...,218,24,0.0,22,39,29,87,24,18,490897
345159,2011-12,T,00,,,State,,,All,TA,...,9552,507,0.2,1564,2294,1572,3352,512,464,6285600
